In [0]:
from pyspark.sql.functions import col, lag, avg, round as spark_round
from pyspark.sql.window import Window

# Step 1: Load Silver Table
silver_df = spark.table("inflation.silver_daily_prices")

# Step 2: Calculate Daily Inflation %
window_spec = Window.partitionBy("item_id").orderBy("date")

gold_df = (
    silver_df
    .withColumn("prev_price", lag("price").over(window_spec))
    .withColumn(
        "daily_inflation_pct",
        spark_round(((col("price") - col("prev_price")) / col("prev_price")) * 100, 2)
    )
)

# Step 3: (Optional) Calculate simple CPI-like index
# For mock, we can average daily_inflation_pct across all items per day
cpi_df = (
    gold_df.groupBy("date")
    .agg(spark_round(avg("daily_inflation_pct"), 2).alias("daily_cpi_index"))
)

# Step 4: Save Gold Tables
gold_df.write.format("delta").mode("overwrite").saveAsTable("inflation.gold_daily_inflation")
cpi_df.write.format("delta").mode("overwrite").saveAsTable("inflation.gold_cpi_index")

# Step 5: Preview Gold Tables
display(spark.table("inflation.gold_daily_inflation").orderBy("date", "item_id"))
display(spark.table("inflation.gold_cpi_index").orderBy("date"))
